![CH4-ADS.png](.\Media\CH4-ADS.png)

## This section contains multiple code blocks pertaining to identifying and diagnosing conditions with databases

## Retrieve database backup status

This query provides a list of databases on this SQL Instance and returns information about their backup history.

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
       -- Retrieve database backup status
       SET NOCOUNT ON
       IF OBJECT_ID('tempdb.dbo.#LastBackupStatus__') IS NOT NULL
        DROP TABLE #LastBackupStatus__
        --
        CREATE TABLE #LastBackupStatus__ (
        DBname nvarchar(256),
        RecoveryModel nvarchar(40),
        IsAgDB nvarchar(40),
        IsPreferredReplica nvarchar(40),
        LastFullBackup datetime,
        LastDiffBackup datetime,
        LastLogBackup datetime,
        DaysSinceLastFull smallint,
        DaysSinceLastDiff smallint,
        MinutesSinceLastLog INT,
        BackupStatus nvarchar(40),
        StatusDescription nvarchar(128))
        --
        INSERT INTO #LastBackupStatus__ (DBname,RecoveryModel,IsAgDB,IsPreferredReplica,LastFullBackup,LastDiffBackup,LastLogBackup,DaysSinceLastFull,DaysSinceLastDiff,MinutesSinceLastLog,BackupStatus,StatusDescription)
    SELECT  db.name AS [DBname]
          , db.recovery_model_desc AS [RecoveryModel]
          , CASE WHEN agdb.database_id IS NOT NULL THEN 'AG Database'
                 ELSE 'Not AG Database'
            END [IsAgDb]
          , CASE WHEN sys.fn_hadr_backup_is_preferred_replica(db.name) = 1
                 THEN 'YES'
                 ELSE 'NO'
            END [IsPreferredReplica]
          , t.LastFullBackup
          , t.LastDiffBackup
          , t.LastLogBackup
          , t.DaysSinceLastFull
          , t.DaysSinceLastDiff
          , t.MinutesSinceLastLog
          , NULL AS [BackupStatus]
          , NULL AS [StatusDescription]
    FROM    sys.databases db
            LEFT OUTER JOIN ( SELECT    p.database_name
                                      , MAX(p.[D]) [LastFullBackup]
                                      , MAX(p.[I]) [LastDiffBackup]
                                      , MAX(p.[L]) [LastLogBackup]
                                      , DATEDIFF(DAY, MAX(p.[D]), GETDATE()) [DaysSinceLastFull]
                                      , DATEDIFF(DAY, MAX(p.[I]), GETDATE()) [DaysSinceLastDiff]
                                      , DATEDIFF(MINUTE, MAX(p.[L]), GETDATE()) [MinutesSinceLastLog]
    
                              FROM      msdb.dbo.backupset bs PIVOT ( MAX(bs.backup_finish_date) FOR [type] IN ( [D], [L], [I] ) )  AS p
                              GROUP BY  p.database_name
                            ) t ON db.name = t.database_name
            LEFT OUTER JOIN sys.dm_hadr_database_replica_states agdb ON agdb.database_id = db.database_id
                                                                  AND agdb.is_local = 1
            WHERE DB.database_id <> 2
    ------ Return data ------
            SELECT
            DBname,
            RecoveryModel,
            IsAgDB,
            IsPreferredReplica,
            LastFullBackup,
            LastDiffBackup,
            LastLogBackup,
            DaysSinceLastFull,
            DaysSinceLastDiff,
            MinutesSinceLastLog,
              CASE
              /* These conditions below will cause a CRITICAL status */
              WHEN [LastFullBackup] IS NULL THEN 'CRITICAL'																					                                                                                                    -- if last_full_backup is null then critical
              WHEN [LastFullBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND [LastDiffBackup] IS NULL AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'CRITICAL'											                    -- if last_full_backup is more than 2 days old and last_differential_backup is null then critical
              WHEN [LastFullBackup] < DATEADD(DD,-7,CURRENT_TIMESTAMP) AND [LastDiffBackup] < DATEADD(DD,-2,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'CRITICAL'				                    -- if last_full_backup is more than 7 days old and last_differential_backup more than 2 days old then critical
              --WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] IS NULL THEN 'CRITICAL'							                                                                                                    -- if recovery_model_desc is SIMPLE and last_tlog_backup is null then critical
              WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] < DATEADD(HH,-6,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'CRITICAL'	                                        -- if last_tlog_backup is more than 6 hours old then critical
              --/* These conditions below will cause a WARNING status */
              --WHEN [LastFullBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND [LastDiffBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND IsAgDB = 'AG Database' AND IsPreferredReplica = 'NO' THEN 'WARNING'				                        -- if last_full_backup is more than 1 day old and last_differential_backup is greater than 1 days old then warning
              WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] < DATEADD(HH,-3,CURRENT_TIMESTAMP) AND isAgDB = 'AG Database' AND isPreferredReplica = 'YES' THEN 'WARNING'	                                                                                                    -- if last_tlog_backup is more than 3 hours old then warning
              /* Everything else will return a GOOD status */
              ELSE 'GOOD'
            END AS BackupStatus,
              CASE
              /* These conditions below will cause a CRITICAL status */
              WHEN [LastFullBackup] IS NULL AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'No FULL backups'																									            -- if last_full_backup is null then critical
              WHEN [LastFullBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND [LastDiffBackup] IS NULL AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'FULL backup > 1 day; no DIFF backups'									-- if last_full_backup is more than 2 days old and last_differential_backup is null then critical
              WHEN [LastFullBackup] < DATEADD(DD,-7,CURRENT_TIMESTAMP) AND [LastDiffBackup] < DATEADD(DD,-2,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'FULL backup > 7 day; DIFF backup > 2 days'	-- if last_full_backup is more than 7 days old and last_differential_backup more than 2 days old then critical
              WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] IS NULL THEN 'No LOG backups'											                                                                                -- if recovery_model_desc is SIMPLE and last_tlog_backup is null then critical
              WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] < DATEADD(HH,-6,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'LOG backup > 6 hours'			                    -- if last_tlog_backup is more than 6 hours old then critical
              --/* These conditions below will cause a WARNING status */
              WHEN [LastFullBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND [LastDiffBackup] < DATEADD(DD,-1,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'FULL backup > 7 day; DIFF backup > 1 day'	-- if last_full_backup is more than 1 day old and last_differential_backup is greater than 1 days old then warning
              WHEN RecoveryModel <> 'SIMPLE' AND DBname <> 'model' AND [LastLogBackup] < DATEADD(HH,-3,CURRENT_TIMESTAMP) AND IsAgDB = 'Not AG Database' AND IsPreferredReplica = 'YES' THEN 'LOG backup > 3 hours'			                    -- if last_tlog_backup is more than 3 hours old then warning
              /* Everything else will return a GOOD status */
              ELSE 'No issues'
            END AS StatusDescription
            FROM
                #LastBackupStatus__
        --
        -- Cleanup
        --
            IF OBJECT_ID('tempdb.dbo.#LastBackupStatus__') IS NOT NULL
                DROP TABLE #LastBackupStatus__

## Last backup information by database

This also provides a list of databases and their backup status, but this is more of an abreviated check then the query above.

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Last backup information by database (Last Backup By Database)
SELECT ISNULL(d.[name], bs.[database_name]) AS [Database], d.recovery_model_desc AS [Recovery Model], 
       d.log_reuse_wait_desc AS [Log Reuse Wait Desc],
    MAX(CASE WHEN [type] = 'D' THEN bs.backup_finish_date ELSE NULL END) AS [Last Full Backup],
    MAX(CASE WHEN [type] = 'I' THEN bs.backup_finish_date ELSE NULL END) AS [Last Differential Backup],
    MAX(CASE WHEN [type] = 'L' THEN bs.backup_finish_date ELSE NULL END) AS [Last Log Backup]
FROM sys.databases AS d WITH (NOLOCK)
LEFT OUTER JOIN msdb.dbo.backupset AS bs WITH (NOLOCK)
ON bs.[database_name] = d.[name] 
AND bs.backup_finish_date > GETDATE()- 30
WHERE d.name <> N'tempdb'
GROUP BY ISNULL(d.[name], bs.[database_name]), d.recovery_model_desc, d.log_reuse_wait_desc, d.[name] 
ORDER BY d.recovery_model_desc, d.[name] OPTION (RECOMPILE);

## Check the Suspect Pages table

The msdb.dbo.suspect\_pages table contains one row per page that has failed with a minor 823 error or an 824 error. Pages are listed in this table because they are suspected of being bad, but they might actually be fine. When a suspect page is repaired, its status is updated in the event\_type column.  

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Look at Suspect Pages table (Suspect Pages)
SELECT DB_NAME(database_id) AS [Database Name], [file_id], page_id, 
       event_type, error_count, last_update_date 
FROM msdb.dbo.suspect_pages WITH (NOLOCK)
ORDER BY database_id OPTION (RECOMPILE);

**event\_type value descriptions**

- 1 = 823 error caused by an operating system CRC error or 824 error other than a bad checksum or a torn page (for example, a bad page ID)
- 2 = Bad checksum
- 3 = Torn page
- 4 = Restored (The page was restored after it was marked bad)
- 5 = Repaired (DBCC repaired the page)
- 7 = Deallocated by DBCC

Ideally, this query returns no results. The table is limited to 1000 rows. If you do get results here, you should do further investigation to determine the root cause

Read this article to lear more about hoe the database engine updates this table. [Manage the suspect\_pages Table](https://bit.ly/2Fvr1c9)

## Retrieve the number of data files in TempDB database

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
EXEC sys.xp_readerrorlog 0, 1, N'The tempdb database has';


Returns the number of data files in the tempdb database

- 4-8 data files that are all the same size is a good starting point
- This query will return no results if your error log has been recycled since the instance was last started

## File names and paths for all user and system databases on instance (Database Filenames and Paths)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- File names and paths for all user and system databases on instance (Database Filenames and Paths)
SELECT DB_NAME([database_id]) AS [Database Name], 
       [file_id], [name], physical_name, [type_desc], state_desc,
	   is_percent_growth, growth, 
	   CONVERT(bigint, growth/128.0) AS [Growth in MB], 
       CONVERT(bigint, size/128.0) AS [Total Size in MB], max_size
FROM sys.master_files WITH (NOLOCK)
ORDER BY DB_NAME([database_id]), [file_id] OPTION (RECOMPILE);

Things to look at:

- Are data files and log files on different drives?
- Is everything on the C: drive?
- Is tempdb on dedicated drives?
- Is there only one tempdb data file?
- Are all of the tempdb data files the same size?
- Are there multiple data files for user databases?
- Is percent growth enabled for any files (which is bad)?

## Drive information for all fixed drives visible to the operating system (Fixed Drives)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Drive information for all fixed drives visible to the operating system (Fixed Drives)
SELECT fixed_drive_path, drive_type_desc, 
CONVERT(DECIMAL(18,2), free_space_in_bytes/1073741824.0) AS [Available Space (GB)]
FROM sys.dm_os_enumerate_fixed_drives WITH (NOLOCK) OPTION (RECOMPILE);

This shows all of your fixed drives, not just LUNs with SQL Server database files

## Volume info for all LUNS that have database files on the current instance (Volume Info)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Volume info for all LUNS that have database files on the current instance (Volume Info)
SELECT DISTINCT vs.volume_mount_point, vs.file_system_type, vs.logical_volume_name, 
CONVERT(DECIMAL(18,2), vs.total_bytes/1073741824.0) AS [Total Size (GB)],
CONVERT(DECIMAL(18,2), vs.available_bytes/1073741824.0) AS [Available Size (GB)],  
CONVERT(DECIMAL(18,2), vs.available_bytes * 1. / vs.total_bytes * 100.) AS [Space Free %],
vs.supports_compression, vs.is_compressed, 
vs.supports_sparse_files, vs.supports_alternate_streams
FROM sys.master_files AS f WITH (NOLOCK)
CROSS APPLY sys.dm_os_volume_stats(f.database_id, f.[file_id]) AS vs 
ORDER BY vs.volume_mount_point OPTION (RECOMPILE);

Shows you the total and free space on the LUNs where you have database files

- Being low on free space can negatively affect performance

[sys.dm\_os\_volume\_stats (Transact-SQL)](https://bit.ly/2oBPNNr)

## Drive level latency information (Drive Level Latency)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Drive level latency information (Drive Level Latency)
SELECT tab.[Drive], tab.volume_mount_point AS [Volume Mount Point], 
	CASE 
		WHEN num_of_reads = 0 THEN 0 
		ELSE (io_stall_read_ms/num_of_reads) 
	END AS [Read Latency],
	CASE 
		WHEN num_of_writes = 0 THEN 0 
		ELSE (io_stall_write_ms/num_of_writes) 
	END AS [Write Latency],
	CASE 
		WHEN (num_of_reads = 0 AND num_of_writes = 0) THEN 0 
		ELSE (io_stall/(num_of_reads + num_of_writes)) 
	END AS [Overall Latency],
	CASE 
		WHEN num_of_reads = 0 THEN 0 
		ELSE (num_of_bytes_read/num_of_reads) 
	END AS [Avg Bytes/Read],
	CASE 
		WHEN num_of_writes = 0 THEN 0 
		ELSE (num_of_bytes_written/num_of_writes) 
	END AS [Avg Bytes/Write],
	CASE 
		WHEN (num_of_reads = 0 AND num_of_writes = 0) THEN 0 
		ELSE ((num_of_bytes_read + num_of_bytes_written)/(num_of_reads + num_of_writes)) 
	END AS [Avg Bytes/Transfer]
FROM (SELECT LEFT(UPPER(mf.physical_name), 2) AS Drive, SUM(num_of_reads) AS num_of_reads,
	         SUM(io_stall_read_ms) AS io_stall_read_ms, SUM(num_of_writes) AS num_of_writes,
	         SUM(io_stall_write_ms) AS io_stall_write_ms, SUM(num_of_bytes_read) AS num_of_bytes_read,
	         SUM(num_of_bytes_written) AS num_of_bytes_written, SUM(io_stall) AS io_stall, vs.volume_mount_point 
      FROM sys.dm_io_virtual_file_stats(NULL, NULL) AS vfs
      INNER JOIN sys.master_files AS mf WITH (NOLOCK)
      ON vfs.database_id = mf.database_id AND vfs.file_id = mf.file_id
	  CROSS APPLY sys.dm_os_volume_stats(mf.database_id, mf.[file_id]) AS vs 
      GROUP BY LEFT(UPPER(mf.physical_name), 2), vs.volume_mount_point) AS tab
ORDER BY [Overall Latency] OPTION (RECOMPILE);

Shows you the drive-level latency for reads and writes, in milliseconds

- Latency above 30-40ms is usually a problem
- These latency numbers include all file activity against all SQL Server database files on each drive since SQL Server was last started

## Calculates average latency per read, per write, and per total input/output for each database file (IO Latency by File)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Calculates average latency per read, per write, and per total input/output for each database file (IO Latency by File)
SELECT DB_NAME(fs.database_id) AS [Database Name], CAST(fs.io_stall_read_ms/(1.0 + fs.num_of_reads) AS NUMERIC(10,1)) AS [avg_read_latency_ms],
CAST(fs.io_stall_write_ms/(1.0 + fs.num_of_writes) AS NUMERIC(10,1)) AS [avg_write_latency_ms],
CAST((fs.io_stall_read_ms + fs.io_stall_write_ms)/(1.0 + fs.num_of_reads + fs.num_of_writes) AS NUMERIC(10,1)) AS [avg_io_latency_ms],
CONVERT(DECIMAL(18,2), mf.size/128.0) AS [File Size (MB)], mf.physical_name, mf.type_desc, fs.io_stall_read_ms, fs.num_of_reads, 
fs.io_stall_write_ms, fs.num_of_writes, fs.io_stall_read_ms + fs.io_stall_write_ms AS [io_stalls], fs.num_of_reads + fs.num_of_writes AS [total_io],
io_stall_queued_read_ms AS [Resource Governor Total Read IO Latency (ms)], io_stall_queued_write_ms AS [Resource Governor Total Write IO Latency (ms)] 
FROM sys.dm_io_virtual_file_stats(null,null) AS fs
INNER JOIN sys.master_files AS mf WITH (NOLOCK)
ON fs.database_id = mf.database_id
AND fs.[file_id] = mf.[file_id]
ORDER BY avg_io_latency_ms DESC OPTION (RECOMPILE);

Helps determine which database files on the entire instance have the most I/O bottlenecks

- This can help you decide whether certain LUNs are overloaded
- Or whether you might want to move some files to a different location or perhaps improve your I/O performance
- These latency numbers include all file activity against each SQL Server database file since SQL Server was last started

## Look for I/O requests taking longer than 15 seconds in the six most recent SQL Server Error Logs (IO Warnings)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Look for I/O requests taking longer than 15 seconds in the six most recent SQL Server Error Logs (IO Warnings)
CREATE TABLE #IOWarningResults(LogDate datetime, ProcessInfo sysname, LogText nvarchar(1000));

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 0, 1, N'taking longer than 15 seconds';

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 1, 1, N'taking longer than 15 seconds';

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 2, 1, N'taking longer than 15 seconds';

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 3, 1, N'taking longer than 15 seconds';

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 4, 1, N'taking longer than 15 seconds';

	INSERT INTO #IOWarningResults 
	EXEC xp_readerrorlog 5, 1, N'taking longer than 15 seconds';

SELECT LogDate, ProcessInfo, LogText
FROM #IOWarningResults
ORDER BY LogDate DESC;

DROP TABLE #IOWarningResults;

Finding 15 second I/O warnings in the SQL Server Error Log is useful evidence of poor I/O performance (which might have many different causes) Look to see if you see any patterns in the results (same files, same drives, same time of day, etc.)

[Diagnostics in SQL Server help detect stalled and stuck I/O operations](https://bit.ly/2qtaw73)

## Get VLF Counts for all databases on the instance (VLF Counts)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get VLF Counts for all databases on the instance (VLF Counts)
SELECT [name] AS [Database Name], [VLF Count]
FROM sys.databases AS db WITH (NOLOCK)
CROSS APPLY (SELECT file_id, COUNT(*) AS [VLF Count]
		     FROM sys.dm_db_log_info(db.database_id)
			 GROUP BY file_id) AS li
ORDER BY [VLF Count] DESC OPTION (RECOMPILE);

High VLF counts can affect write performance to the log file, and they can make full database restores and crash recovery take much longer. Try to keep your VLF counts under 200 in most cases (depending on log file size)

[Important change to VLF creation algorithm in SQL Server 2014](https://bit.ly/2Hsjbg4)

[SQL Server Transaction Log Architecture and Management Guide](https://bit.ly/2JjmQRZ)

## Get CPU utilization by database (CPU Usage by Database)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get CPU utilization by database (CPU Usage by Database)
WITH DB_CPU_Stats
AS
(SELECT pa.DatabaseID, DB_Name(pa.DatabaseID) AS [Database Name], SUM(qs.total_worker_time/1000) AS [CPU_Time_Ms]
 FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
 CROSS APPLY (SELECT CONVERT(int, value) AS [DatabaseID] 
              FROM sys.dm_exec_plan_attributes(qs.plan_handle)
              WHERE attribute = N'dbid') AS pa
 GROUP BY DatabaseID)
SELECT ROW_NUMBER() OVER(ORDER BY [CPU_Time_Ms] DESC) AS [CPU Rank],
       [Database Name], [CPU_Time_Ms] AS [CPU Time (ms)], 
       CAST([CPU_Time_Ms] * 1.0 / SUM([CPU_Time_Ms]) OVER() * 100.0 AS DECIMAL(5, 2)) AS [CPU Percent]
FROM DB_CPU_Stats
WHERE DatabaseID <> 32767 -- ResourceDB
ORDER BY [CPU Rank] OPTION (RECOMPILE);

Helps determine which database is using the most CPU resources on the instance  
**Note:** This only reflects CPU usage from the currently cached query plans

## Get I/O utilization by database (IO Usage By Database)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get I/O utilization by database (IO Usage By Database)
WITH Aggregate_IO_Statistics
AS (SELECT DB_NAME(database_id) AS [Database Name],
    CAST(SUM(num_of_bytes_read + num_of_bytes_written) / 1048576 AS DECIMAL(12, 2)) AS [ioTotalMB],
    CAST(SUM(num_of_bytes_read ) / 1048576 AS DECIMAL(12, 2)) AS [ioReadMB],
    CAST(SUM(num_of_bytes_written) / 1048576 AS DECIMAL(12, 2)) AS [ioWriteMB]
    FROM sys.dm_io_virtual_file_stats(NULL, NULL) AS [DM_IO_STATS]
    GROUP BY database_id)
SELECT ROW_NUMBER() OVER (ORDER BY ioTotalMB DESC) AS [I/O Rank],
        [Database Name], ioTotalMB AS [Total I/O (MB)],
        CAST(ioTotalMB / SUM(ioTotalMB) OVER () * 100.0 AS DECIMAL(5, 2)) AS [Total I/O %],
        ioReadMB AS [Read I/O (MB)], 
		CAST(ioReadMB / SUM(ioReadMB) OVER () * 100.0 AS DECIMAL(5, 2)) AS [Read I/O %],
        ioWriteMB AS [Write I/O (MB)], 
		CAST(ioWriteMB / SUM(ioWriteMB) OVER () * 100.0 AS DECIMAL(5, 2)) AS [Write I/O %]
FROM Aggregate_IO_Statistics
ORDER BY [I/O Rank] OPTION (RECOMPILE);

Helps determine which database is using the most I/O resources on the instance

- These numbers are cumulative since the last service restart
- They include all I/O activity, not just the nominal I/O workload

## Get total buffer usage by database for current instance (Total Buffer Usage by Database)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get total buffer usage by database for current instance (Total Buffer Usage by Database)
-- This make take some time to run on a busy instance
WITH AggregateBufferPoolUsage
AS
(SELECT DB_NAME(database_id) AS [Database Name],
CAST(COUNT(*) * 8/1024.0 AS DECIMAL (10,2))  AS [CachedSize]
FROM sys.dm_os_buffer_descriptors WITH (NOLOCK)
WHERE database_id <> 32767 -- ResourceDB
GROUP BY DB_NAME(database_id))
SELECT ROW_NUMBER() OVER(ORDER BY CachedSize DESC) AS [Buffer Pool Rank], [Database Name], CachedSize AS [Cached Size (MB)],
       CAST(CachedSize / SUM(CachedSize) OVER() * 100.0 AS DECIMAL(5,2)) AS [Buffer Pool Percent]
FROM AggregateBufferPoolUsage
ORDER BY [Buffer Pool Rank] OPTION (RECOMPILE);

Tells you how much memory (in the buffer pool) is being used by each database on the instance

## Get tempdb version store space usage by database (Version Store Space Usage)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get tempdb version store space usage by database (Version Store Space Usage)
SELECT DB_NAME(database_id) AS [Database Name],
       reserved_page_count AS [Version Store Reserved Page Count], 
	   reserved_space_kb/1024 AS [Version Store Reserved Space (MB)] 
FROM sys.dm_tran_version_store_space_usage WITH (NOLOCK) 
ORDER BY reserved_space_kb/1024 DESC OPTION (RECOMPILE);

[sys.dm\_tran\_version\_store\_space\_usage (Transact-SQL)](https://bit.ly/2vh3Bmk)

## Get top total logical reads queries for entire instance (Top Logical Reads Queries)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get top total logical reads queries for entire instance (Top Logical Reads Queries)
SELECT TOP(50) DB_NAME(t.[dbid]) AS [Database Name],
REPLACE(REPLACE(LEFT(t.[text], 255), CHAR(10),''), CHAR(13),'') AS [Short Query Text], 
qs.total_logical_reads AS [Total Logical Reads],
qs.min_logical_reads AS [Min Logical Reads],
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads],
qs.max_logical_reads AS [Max Logical Reads],   
qs.min_worker_time AS [Min Worker Time],
qs.total_worker_time/qs.execution_count AS [Avg Worker Time], 
qs.max_worker_time AS [Max Worker Time], 
qs.min_elapsed_time AS [Min Elapsed Time], 
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time], 
qs.max_elapsed_time AS [Max Elapsed Time],
qs.execution_count AS [Execution Count], 
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) COLLATE Latin1_General_BIN2 LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],
qs.creation_time AS [Creation Time]
--,t.[text] AS [Complete Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp 
ORDER BY qs.total_logical_reads DESC OPTION (RECOMPILE);

Helps you find the most expensive queries from a memory perspective across the entire instance

- Can also help track down parameter sniffing issues

## Get top average elapsed time queries for entire instance (Top Avg Elapsed Time Queries)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get top average elapsed time queries for entire instance (Top Avg Elapsed Time Queries)
SELECT TOP(50) DB_NAME(t.[dbid]) AS [Database Name], 
REPLACE(REPLACE(LEFT(t.[text], 255), CHAR(10),''), CHAR(13),'') AS [Short Query Text],  
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time],
qs.min_elapsed_time, qs.max_elapsed_time, qs.last_elapsed_time,
qs.execution_count AS [Execution Count],  
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads], 
qs.total_physical_reads/qs.execution_count AS [Avg Physical Reads], 
qs.total_worker_time/qs.execution_count AS [Avg Worker Time],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) COLLATE Latin1_General_BIN2 LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],
qs.creation_time AS [Creation Time]
--,t.[text] AS [Complete Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp 
ORDER BY qs.total_elapsed_time/qs.execution_count DESC OPTION (RECOMPILE);

Helps you find the highest average elapsed time queries across the entire instance

- Can also help track down parameter sniffing issues

## Look at UDF execution statistics (UDF Stats by DB)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Look at UDF execution statistics (UDF Stats by DB)
SELECT TOP (25) DB_NAME(database_id) AS [Database Name], 
		   OBJECT_NAME(object_id, database_id) AS [Function Name],
		   total_worker_time, execution_count, total_elapsed_time,  
           total_elapsed_time/execution_count AS [avg_elapsed_time],  
           last_elapsed_time, last_execution_time, cached_time, [type_desc] 
FROM sys.dm_exec_function_stats WITH (NOLOCK) 
ORDER BY total_worker_time DESC OPTION (RECOMPILE);

[sys.dm\_exec\_function\_stats (Transact-SQL)](https://bit.ly/2q1Q6BM)

[Showplan Enhancements for UDFs](https://bit.ly/2LVqiQ1)

# Database specific queries

> **Note**: Please switch to a user database that you are interested in!

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- **** Please switch to a user database that you are interested in! *****
--USE YourDatabaseName; -- make sure to change to an actual database on your instance, not the master system database
--GO

## Individual File Sizes and space available for current database (File Sizes and Space)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Individual File Sizes and space available for current database (File Sizes and Space)
SELECT f.name AS [File Name] , f.physical_name AS [Physical Name], 
CAST((f.size/128.0) AS DECIMAL(15,2)) AS [Total Size in MB],
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, 'SpaceUsed') AS int)/128.0 AS DECIMAL(15,2)) 
AS [Available Space In MB],
CAST((f.size/128.0) AS DECIMAL(15,2)) - 
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, 'SpaceUsed') AS int)/128.0 AS DECIMAL(15,2)) AS [Used Space in MB],
f.[file_id], fg.name AS [Filegroup Name],
f.is_percent_growth, f.growth, fg.is_default, fg.is_read_only, 
fg.is_autogrow_all_files
FROM sys.database_files AS f WITH (NOLOCK) 
LEFT OUTER JOIN sys.filegroups AS fg WITH (NOLOCK)
ON f.data_space_id = fg.data_space_id
ORDER BY f.[file_id] OPTION (RECOMPILE);

Look at how large and how full the files are and where they are located

- Make sure the transaction log is not full!!
- is\_autogrow\_all\_files was new for SQL Server 2016. Equivalent to TF 1117 for user databases

[SQL Server 2016: Changes in default behavior for autogrow and allocations for tempdb and user databases](https://bit.ly/2evRZSR)

## Log space usage for current database (Log Space Usage)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Log space usage for current database  (Log Space Usage)
SELECT DB_NAME(lsu.database_id) AS [Database Name], db.recovery_model_desc AS [Recovery Model],
		CAST(lsu.total_log_size_in_bytes/1048576.0 AS DECIMAL(10, 2)) AS [Total Log Space (MB)],
		CAST(lsu.used_log_space_in_bytes/1048576.0 AS DECIMAL(10, 2)) AS [Used Log Space (MB)], 
		CAST(lsu.used_log_space_in_percent AS DECIMAL(10, 2)) AS [Used Log Space %],
		CAST(lsu.log_space_in_bytes_since_last_backup/1048576.0 AS DECIMAL(10, 2)) AS [Used Log Space Since Last Backup (MB)],
		db.log_reuse_wait_desc		 
FROM sys.dm_db_log_space_usage AS lsu WITH (NOLOCK)
INNER JOIN sys.databases AS db WITH (NOLOCK)
ON lsu.database_id = db.database_id
OPTION (RECOMPILE);

Look at log file size and usage, along with the log reuse wait description for the current database

[sys.dm\_db\_log\_space\_usage (Transact-SQL)](https://bit.ly/2H4MQw9)

## Status of last VLF for current database (Last VLF Status)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Status of last VLF for current database (Last VLF Status)
SELECT TOP(1) DB_NAME(li.database_id) AS [Database Name], li.[file_id],
              li.vlf_size_mb, li.vlf_sequence_number, li.vlf_active, li.vlf_status
FROM sys.dm_db_log_info(DB_ID()) AS li 
ORDER BY vlf_sequence_number DESC OPTION (RECOMPILE);

Determine whether you will be able to shrink the transaction log file

vlf\_status Values

- 0 is inactive
- 1 is initialized but unused
- 2 is active

[sys.dm\_db\_log\_info (Transact-SQL)](https://bit.ly/2EQUU1v)

## Get database scoped configuration values for current database (Database-scoped Configurations)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get database scoped configuration values for current database (Database-scoped Configurations)
SELECT configuration_id, name, [value] AS [value_for_primary], value_for_secondary
FROM sys.database_scoped_configurations WITH (NOLOCK) OPTION (RECOMPILE);

This lets you see the value of these new properties for the current database

Clear plan cache for current database:

- `ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;`

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;

[ALTER DATABASE SCOPED CONFIGURATION (Transact-SQL)](https://bit.ly/2sOH7nb)

## I/O Statistics by file for the current database (IO Stats By File)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- I/O Statistics by file for the current database  (IO Stats By File)
SELECT DB_NAME(DB_ID()) AS [Database Name], df.name AS [Logical Name], vfs.[file_id], df.type_desc,
df.physical_name AS [Physical Name], CAST(vfs.size_on_disk_bytes/1048576.0 AS DECIMAL(10, 2)) AS [Size on Disk (MB)],
vfs.num_of_reads, vfs.num_of_writes, vfs.io_stall_read_ms, vfs.io_stall_write_ms,
CAST(100. * vfs.io_stall_read_ms/(vfs.io_stall_read_ms + vfs.io_stall_write_ms) AS DECIMAL(10,1)) AS [IO Stall Reads Pct],
CAST(100. * vfs.io_stall_write_ms/(vfs.io_stall_write_ms + vfs.io_stall_read_ms) AS DECIMAL(10,1)) AS [IO Stall Writes Pct],
(vfs.num_of_reads + vfs.num_of_writes) AS [Writes + Reads], 
CAST(vfs.num_of_bytes_read/1048576.0 AS DECIMAL(10, 2)) AS [MB Read], 
CAST(vfs.num_of_bytes_written/1048576.0 AS DECIMAL(10, 2)) AS [MB Written],
CAST(100. * vfs.num_of_reads/(vfs.num_of_reads + vfs.num_of_writes) AS DECIMAL(10,1)) AS [# Reads Pct],
CAST(100. * vfs.num_of_writes/(vfs.num_of_reads + vfs.num_of_writes) AS DECIMAL(10,1)) AS [# Write Pct],
CAST(100. * vfs.num_of_bytes_read/(vfs.num_of_bytes_read + vfs.num_of_bytes_written) AS DECIMAL(10,1)) AS [Read Bytes Pct],
CAST(100. * vfs.num_of_bytes_written/(vfs.num_of_bytes_read + vfs.num_of_bytes_written) AS DECIMAL(10,1)) AS [Written Bytes Pct]
FROM sys.dm_io_virtual_file_stats(DB_ID(), NULL) AS vfs
INNER JOIN sys.database_files AS df WITH (NOLOCK)
ON vfs.[file_id]= df.[file_id] OPTION (RECOMPILE);

This helps you characterize your workload better from an I/O perspective for this database

- It helps you determine whether you have an OLTP or DW/DSS type of workload

## Get most frequently executed queries for this database (Query Execution Counts)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get most frequently executed queries for this database (Query Execution Counts)
SELECT TOP(50) LEFT(t.[text], 50) AS [Short Query Text], qs.execution_count AS [Execution Count],
qs.total_logical_reads AS [Total Logical Reads],
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads],
qs.total_worker_time AS [Total Worker Time],
qs.total_worker_time/qs.execution_count AS [Avg Worker Time], 
qs.total_elapsed_time AS [Total Elapsed Time],
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) COLLATE Latin1_General_BIN2 LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],
qs.creation_time AS [Creation Time]
--,t.[text] AS [Complete Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp 
WHERE t.dbid = DB_ID()
ORDER BY qs.execution_count DESC OPTION (RECOMPILE);

Tells you which cached queries are called the most often

- This helps you characterize and baseline your workload
- It also helps you find possible caching opportunities

## Get lock waits for current database (Lock Waits)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get lock waits for current database (Lock Waits)
SELECT o.name AS [table_name], i.name AS [index_name], ios.index_id, ios.partition_number,
		SUM(ios.row_lock_wait_count) AS [total_row_lock_waits], 
		SUM(ios.row_lock_wait_in_ms) AS [total_row_lock_wait_in_ms],
		SUM(ios.page_lock_wait_count) AS [total_page_lock_waits],
		SUM(ios.page_lock_wait_in_ms) AS [total_page_lock_wait_in_ms],
		SUM(ios.page_lock_wait_in_ms)+ SUM(row_lock_wait_in_ms) AS [total_lock_wait_in_ms]
FROM sys.dm_db_index_operational_stats(DB_ID(), NULL, NULL, NULL) AS ios
INNER JOIN sys.objects AS o WITH (NOLOCK)
ON ios.[object_id] = o.[object_id]
INNER JOIN sys.indexes AS i WITH (NOLOCK)
ON ios.[object_id] = i.[object_id] 
AND ios.index_id = i.index_id
WHERE o.[object_id] > 100
GROUP BY o.name, i.name, ios.index_id, ios.partition_number
HAVING SUM(ios.page_lock_wait_in_ms)+ SUM(row_lock_wait_in_ms) > 0
ORDER BY total_lock_wait_in_ms DESC OPTION (RECOMPILE);

This query is helpful for troubleshooting blocking and deadlocking issues

## Look at UDF execution statistics (UDF Statistics)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Look at UDF execution statistics (UDF Statistics)
SELECT OBJECT_NAME(object_id) AS [Function Name], execution_count,
	   total_worker_time, total_logical_reads, total_physical_reads, total_elapsed_time, 
	   total_elapsed_time/execution_count AS [avg_elapsed_time],
	   FORMAT(cached_time, 'yyyy-MM-dd HH:mm:ss', 'en-US') AS [Plan Cached Time]
FROM sys.dm_exec_function_stats WITH (NOLOCK) 
WHERE database_id = DB_ID()
ORDER BY total_worker_time DESC OPTION (RECOMPILE); 

New for SQL Server 2016

- Helps you investigate scalar UDF performance issues
- Does not return information for table valued functions

[sys.dm\_exec\_function\_stats (Transact-SQL)](https://bit.ly/2q1Q6BM)

## Determine which scalar UDFs are in-lineable (Inlineable UDFs)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Determine which scalar UDFs are in-lineable (Inlineable UDFs)
SELECT OBJECT_NAME(m.object_id) AS [Function Name], is_inlineable, inline_type
FROM sys.sql_modules AS m WITH (NOLOCK) 
LEFT OUTER JOIN sys.dm_exec_function_stats AS efs WITH (NOLOCK)
ON  m.object_id = efs.object_id
WHERE efs.type_desc = N'SQL_SCALAR_FUNCTION'
OPTION (RECOMPILE);

[Scalar UDF Inlining](https://bit.ly/2JU971M)

[sys.sql\_modules (Transact-SQL)](https://bit.ly/2Qt216S)

## Get QueryStore Options for this database (QueryStore Options)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get QueryStore Options for this database (QueryStore Options)
SELECT actual_state_desc, desired_state_desc, [interval_length_minutes],
       current_storage_size_mb, [max_storage_size_mb], 
	   query_capture_mode_desc, size_based_cleanup_mode_desc
FROM sys.database_query_store_options WITH (NOLOCK) OPTION (RECOMPILE);

New for SQL Server 2016

- Requires that Query Store is enabled for this database
- Make sure that the actual\_state\_desc is the same as desired\_state\_desc
- Make sure that the current\_storage\_size\_mb is less than the max\_storage\_size\_mb

[Tuning Workload Performance with Query Store](https://bit.ly/1kHSl7w)

## Get input buffer information for the current database (Input Buffer)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get input buffer information for the current database (Input Buffer)
SELECT es.session_id, DB_NAME(es.database_id) AS [Database Name],
       es.login_time, es.cpu_time, es.logical_reads, es.memory_usage,
       es.[status], ib.event_info AS [Input Buffer]
FROM sys.dm_exec_sessions AS es WITH (NOLOCK)
CROSS APPLY sys.dm_exec_input_buffer(es.session_id, NULL) AS ib
WHERE es.database_id = DB_ID()
AND es.session_id > 50
AND es.session_id <> @@SPID OPTION (RECOMPILE);

Gives you input buffer information from all non-system sessions for the current database

- Replaces `DBCC INPUTBUFFER`

[New DMF for retrieving input buffer in SQL Serve](https://bit.ly/2uHKMbz)r

[sys.dm\_exec\_input\_buffer (Transact-SQL)](https://bit.ly/2J5Hf9q)

## Get database automatic tuning options (Automatic Tuning Options)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get database automatic tuning options (Automatic Tuning Options)
SELECT [name], desired_state_desc, actual_state_desc, reason_desc
FROM sys.database_automatic_tuning_options WITH (NOLOCK)
OPTION (RECOMPILE);

[sys.database\_automatic\_tuning\_options (Transact-SQL)](https://bit.ly/2FHhLkL)

## Look at recent Full backups for the current database (Recent Full Backups)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Look at recent Full backups for the current database (Recent Full Backups)
SELECT TOP (30) bs.machine_name, bs.server_name, bs.database_name AS [Database Name], bs.recovery_model,
CONVERT (BIGINT, bs.backup_size / 1048576 ) AS [Uncompressed Backup Size (MB)],
CONVERT (BIGINT, bs.compressed_backup_size / 1048576 ) AS [Compressed Backup Size (MB)],
CONVERT (NUMERIC (20,2), (CONVERT (FLOAT, bs.backup_size) /
CONVERT (FLOAT, bs.compressed_backup_size))) AS [Compression Ratio], bs.has_backup_checksums, bs.is_copy_only, bs.encryptor_type,
DATEDIFF (SECOND, bs.backup_start_date, bs.backup_finish_date) AS [Backup Elapsed Time (sec)],
bs.backup_finish_date AS [Backup Finish Date], bmf.physical_device_name AS [Backup Location], bmf.physical_block_size
FROM msdb.dbo.backupset AS bs WITH (NOLOCK)
INNER JOIN msdb.dbo.backupmediafamily AS bmf WITH (NOLOCK)
ON bs.media_set_id = bmf.media_set_id  
WHERE bs.database_name = DB_NAME(DB_ID())
AND bs.[type] = 'D' -- Change to L if you want Log backups
ORDER BY bs.backup_finish_date DESC OPTION (RECOMPILE);

Things to look at:

- Are your backup sizes and times changing over time?
- Are you using backup compression?
- Are you using backup checksums?
- Are you doing copy\_only backups?
- Are you doing encrypted backups?
- Have you done any backup tuning with striped backups, or changing the parameters of the backup command?
- Where are the backups going to?

In SQL Server 2016, native SQL Server backup compression [actually works much better](https://bit.ly/28Rpb2x) with databases that are using TDE than in previous versions

## Retrieve last database integrity check (DBCC CHECKDB)

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Retrieve last integrity check date
DECLARE @SQLcmd nvarchar(MAX)
    IF CAST((SERVERPROPERTY('productmajorversion')) AS smallint) < 12 --For SQL Server 2012 and below, use DBCC DBINFO to retrieve last good CheckDBstatus
BEGIN
        SET @SQLcmd =
        'SET NOCOUNT ON
        DECLARE @DatabaseName nvarchar(512)
        DECLARE @SQLcmd nvarchar(MAX)
        DECLARE @LastCheckDBdays smallint
        SET @LastCheckDBdays = 7
        --
        --
        IF OBJECT_ID(''tempdb.dbo.#Databases__'') IS NOT NULL
            DROP TABLE #Databases__
        --
        IF OBJECT_ID(''tempdb.dbo.#LastDBCC__'') IS NOT NULL
            DROP TABLE #LastDBCC__
        --
        IF OBJECT_ID(''tempdb.dbo.#DBinfoResults__'') IS NOT NULL
            DROP TABLE #DBinfoResults__

        --
        CREATE TABLE #Databases__ (DatabaseName nvarchar(512) NULL,isProcessed BIT)
        CREATE TABLE #DBinfoResults__ (ParentObject nvarchar(512),[Object] nvarchar(1000),[Field] nvarchar(512),[Value] nvarchar(1000))
        CREATE TABLE #LastDBCC__ (DatabaseName nvarchar(512),LastGoodCheckDB datetime,CheckDBstatus nvarchar(8))
        --
            INSERT INTO #Databases__ (DatabaseName,isProcessed)
            SELECT
                [name],
                0 AS isProcessed
            FROM
                    sys.databases
            WHERE
                    [name] NOT IN (''tempdb'')
        --
            WHILE EXISTS (SELECT TOP 1 DatabaseName FROM #Databases__ WHERE isProcessed = 0)
                BEGIN
                    SET @DatabaseName = (SELECT TOP 1 DatabaseName FROM #Databases__ WHERE isProcessed = 0)
                    SET @SQLcmd = ''
                        INSERT INTO #DBinfoResults__ ([ParentObject],[Object],[Field],[Value])
                        EXEC(''DBCC DBINFO([''+@DatabaseName+'']) WITH TABLERESULTS'')''
                -- PRINT @SQLcmd
                EXEC (@SQLcmd)
                    --
                    INSERT INTO #LastDBCC__ (DatabaseName,LastGoodCheckDB,CheckDBstatus)
                        SELECT @DatabaseName AS DatabaseName,[Value] AS LastGoodCheckDB, NULL AS CheckDBstatus
                        FROM #DBinfoResults__ 
                        WHERE [Field] = ''dbi_dbccLastKnownGood''
                    --
                    TRUNCATE TABLE #DBinfoResults__
					--
                    UPDATE #Databases__ SET isProcessed = 1 WHERE DatabaseName = @DatabaseName
            END
            --
            -- Return results
            --
                SELECT
                DatabaseName,
                LastGoodCheckDB,
                            CASE
                                WHEN [LastGoodCheckDB] < DATEADD(DD,@LastCheckDBdays,CURRENT_TIMESTAMP) THEN ''CRITICAL''
                                WHEN [LastGoodCheckDB] > DATEADD(DD,@LastCheckDBdays,CURRENT_TIMESTAMP) THEN ''GOOD''
                            END AS CheckDBstatus 
                FROM
                    #LastDBCC__
            --
            -- Cleanup
            --
                IF OBJECT_ID(''tempdb.dbo.#Databases__'') IS NOT NULL
                    DROP TABLE #Databases__
            --
                IF OBJECT_ID(''tempdb.dbo.#DBinfoResults__'') IS NOT NULL
                    DROP TABLE #DBinfoResults__
            --
                IF OBJECT_ID(''tempdb.dbo.#LastDBCC__'') IS NOT NULL
                    DROP TABLE #LastDBCC__'
          -- PRINT @SQLcmd
    END
ELSE
    BEGIN
        SET @SQLcmd = 
            'SET NOCOUNT ON
            DECLARE @SQLcmd nvarchar(MAX)
            DECLARE @LastCheckDBdays smallint
            SET @LastCheckDBdays = 7
            --
            IF OBJECT_ID(''tempdb.dbo.#StatementResults'') IS NOT NULL
                DROP TABLE #StatementResults
            IF OBJECT_ID(''tempdb.dbo.#LastGoodCheckDBstatements'') IS NOT NULL
                DROP TABLE #LastGoodCheckDBstatements
                --
                CREATE TABLE #StatementResults (DatabaseName nvarchar(128) NULL, LastDBCCtime sql_variant NULL)
                --
            SELECT
                    ''INSERT INTO #StatementResults (DatabaseName, LastDBCCtime) SELECT ''''''+[name]+'''''' AS DatabaseName, DATABASEPROPERTYEX (''''''+[name]+'''''' , ''''LastGoodCheckDbTime'''')'' AS LastGoodCheckDBstatement
                    ,0 AS isProcessed
                INTO
                    #LastGoodCheckDBstatements
                FROM
                    sys.databases
                WHERE
                    [name] NOT IN (''tempdb'')

                    WHILE EXISTS(SELECT TOP 1 LastGoodCheckDBstatement FROM #LastGoodCheckDBstatements WHERE isProcessed = 0)
            BEGIN
                    SET @SQLcmd = (SELECT TOP 1 LastGoodCheckDBstatement FROM #LastGoodCheckDBstatements WHERE isProcessed = 0)
                    EXEC sp_executeSQL @SQLCMD
                --
                UPDATE #LastGoodCheckDBstatements SET isProcessed = 1 WHERE LastGoodCheckDBstatement = @SQLcmd

            END	
            --
            -- Results
            --
                SELECT
                    DatabaseName,
                    LastDBCCtime AS LastGoodCheckDB,
                        CASE
                            WHEN LastDBCCtime <  DATEADD(DD,@LastCheckDBdays,CURRENT_TIMESTAMP) THEN ''CRITICAL''
                            WHEN LastDBCCtime >  DATEADD(DD,@LastCheckDBdays,CURRENT_TIMESTAMP) THEN ''GOOD''
                        END AS CheckDBstatus
                FROM
                    #StatementResults'
    END
            EXEC (@SQLcmd)
            --PRINT @SQLcmd
